In [81]:
def viterbi_enc(fsm,init_st,x):
    code_stream = []
    ps = init_st
    for c in x:
        code_stream.append(fsm[ps][c]['out'])
        ns = fsm[ps][c]['ns']
        ps = ns
    return code_stream

def viterbi_dec(fsm_inv,code_stream):
    dp_list = []
    d = {}
    for st in fsm_inv.keys():
        d[st] = (0,st)     
    dp_list.append(d)
    
    for code in code_stream:
        d = {}
        for st in fsm_inv.keys():
            d[st] = min([(hamming_distance(code,fsm_inv[st][src_st]['out']) +
                             dp_list[-1][src_st][0],src_st) for src_st in fsm_inv[st].keys()])  
            
        dp_list.append(d)
    decoded_msg = ''
    st = min([(dp_list[-1][k][0],k) for k in dp_list[-1].keys()])[1]
    for i in range(len(dp_list)-1,0,-1):
        src_st = dp_list[i][st][1]
        decoded_msg = fsm_inv[st][src_st]['in'] + decoded_msg
        st = src_st
    return decoded_msg

def hamming_distance(s1,s2):
    dist = 0
    for i in range(len(s1)):
        if not s1[i] == s2[i]:
            dist+=1
    return dist

In [82]:
fsm = {'a':{'0':{'ns':'a','out':'00'},
            '1':{'ns':'b','out':'11'}},
       'b':{'0':{'ns':'c','out':'10'},
            '1':{'ns':'d','out':'01'}},
       'c':{'0':{'ns':'a','out':'11'},
            '1':{'ns':'b','out':'00'}},
       'd':{'0':{'ns':'c','out':'01'},
            '1':{'ns':'d','out':'10'}}
      }
fsm_inv = {'a':{'a':{'in':'0', 'out':'00'},
                'c':{'in':'0', 'out':'11'}},
           'b':{'a':{'in':'1', 'out':'11'},
                'c':{'in':'1', 'out':'11'}},
           'c':{'b':{'in':'0', 'out':'10'},
                'd':{'in':'0', 'out':'01'}},
           'd':{'b':{'in':'1', 'out':'01'},
                'd':{'in':'1', 'out':'10'}}
          }

In [86]:
code_stream = viterbi_enc(fsm,'a','10011')

In [84]:
msg = viterbi_dec(fsm_inv,code_stream)
msg

'10011'